In [6]:
from datetime import datetime

import sqlalchemy
from sqlalchemy import create_engine, select, func, update, text, Column, Integer, Numeric, String, DateTime, ForeignKey, Boolean
from sqlalchemy.orm import sessionmaker, declarative_base, relationship, backref

In [7]:
engine = create_engine('sqlite:///:memory:')

Base = declarative_base()


class Cookie(Base):
    __tablename__ = 'cookies'

    cookie_id = Column(Integer, primary_key=True)
    cookie_name = Column(String(50), index=True)
    cookie_recipe_url = Column(String(255))
    cookie_sku = Column(String(55))
    quantity = Column(Integer())
    unit_cost = Column(Numeric(12, 2))
    
    def __repr__(self):
        return "Cookie(cookie_name='{self.cookie_name}', " \
                       "cookie_recipe_url='{self.cookie_recipe_url}', " \
                       "cookie_sku='{self.cookie_sku}', " \
                       "quantity={self.quantity}, " \
                       "unit_cost={self.unit_cost})".format(self=self)
    

class OrderType(Base):
    __tablename__ = 'ordertype'
    ordertype_id = Column(Integer(), primary_key=True)
    item_sku = Column(String(55), ForeignKey('cookies.cookie_sku'))
    item_type = Column(String(55))
    
    def __repr__(self):
        return "Order(item_sku={self.item_sku}, " \
                      "item_type={self.item_type}".format(self=self)

    
Base.metadata.create_all(engine)



new_session = sessionmaker()
session = new_session(bind=engine.connect(), join_transaction_mode="create_savepoint")



dcc = Cookie(cookie_name='dark chocolate chip',
            cookie_recipe_url='http://some.aweso.me/cookie/recipe_dark.html',
            cookie_sku='CC02',
            quantity=1,
            unit_cost=0.75)

mol = Cookie(cookie_name='molasses',
            cookie_recipe_url='http://some.aweso.me/cookie/recipe_molasses.html',
            cookie_sku='MOL01',
            quantity=1,
            unit_cost=0.80)

session.add(dcc)
session.add(mol)


o1 = OrderType(item_sku='CC02', item_type='Cookie - Dessert')
o2 = OrderType(item_sku='MOL01', item_type='Cookie - Desser')
session.add(o1)
session.add(o2)
session.flush()

print(
    session.scalars(
            select(func.count())
            .select_from(Cookie)
            .where(Cookie.cookie_sku=='MOL01')
        ).one()
)

# with session.begin() as sess:

#     dcc = Cookie(cookie_name='dark chocolate chip',
#                 cookie_recipe_url='http://some.aweso.me/cookie/recipe_dark.html',
#                 cookie_sku='CC02',
#                 quantity=1,
#                 unit_cost=0.75)

#     mol = Cookie(cookie_name='molasses',
#                 cookie_recipe_url='http://some.aweso.me/cookie/recipe_molasses.html',
#                 cookie_sku='MOL01',
#                 quantity=1,
#                 unit_cost=0.80)
#     sess.add(dcc)
#     sess.add(mol)
#     sess.flush()

#     print(
#         sess.scalars(
#                 select(func.count())
#                 .select_from(Cookie)
#                 .where(Cookie.cookie_sku=='MOL01')
#             ).one()
#     )



1


In [8]:
print(
    session.scalars(
        select(func.count())
        .select_from(Cookie)
    ).one()
)

2


In [5]:
from sqlalchemy.orm import load_only, joinedload

print(
    select(Cookie).options(
        load_only(Cookie.cookie_id, Cookie.cookie_name)),
        joinedload(Cookie.cookie_sku)
)

SELECT cookies.cookie_id, cookies.cookie_name 
FROM cookies Load(Mapper[Cookie(cookies)])


In [10]:
import pandas as pd
from pandas import read_sql

read_sql(session.execute(text("SELECT * FROM cookies")), engine)

TypeError: __init__() got multiple values for argument 'schema'

In [3]:
print(Base.metadata.tables)

FacadeDict({'cookies': Table('cookies', MetaData(), Column('cookie_id', Integer(), table=<cookies>, primary_key=True, nullable=False), Column('cookie_name', String(length=50), table=<cookies>), Column('cookie_recipe_url', String(length=255), table=<cookies>), Column('cookie_sku', String(length=55), table=<cookies>), Column('quantity', Integer(), table=<cookies>), Column('unit_cost', Numeric(precision=12, scale=2), table=<cookies>), schema=None), 'ordertype': Table('ordertype', MetaData(), Column('ordertype_id', Integer(), table=<ordertype>, primary_key=True, nullable=False), Column('item_sku', String(length=55), ForeignKey('cookies.cookie_sku'), table=<ordertype>), Column('item_type', String(length=55), table=<ordertype>), schema=None)})


In [80]:
engine.connect().execute(
    text("UPDATE cookies SET cookie_sku='ZZZ123' WHERE cookie_sku='CC02'")
)

In [9]:
q0 = (
    select(Cookie.cookie_id, Cookie.cookie_name)
)
q = session.execute(q0.filter(Cookie.cookie_name=="dark chocolate chip"))

print(q)

In [28]:
q00 = session.execute(q0).merge().fetchall()
print(q00)

[(1, 'dark chocolate chip'), (2, 'molasses')]


In [29]:
d = dict(q00)
print(d)

{1: 'dark chocolate chip', 2: 'molasses'}


In [31]:
q1 = (
    select(Cookie.cookie_id, Cookie.cookie_name, Cookie.quantity)
)

q11 = session.execute(q1).merge().fetchall()

d1 = dict(map(
    lambda _t: ((_t[1], 'True'), _t[2]),
    q11
))

print(d1)

{('dark chocolate chip', 'True'): 1, ('molasses', 'True'): 1}


In [43]:
import pandas as pd

q4 = (select(Cookie.cookie_id, Cookie.cookie_name, Cookie.quantity))
q44 = session.execute(q4)

df = pd.DataFrame(q44, columns=["cookie_id", "cookie_name", "cookie_quantity"])

df

,cookie_id,cookie_name,cookie_quantity
0,1,dark chocolate chip,1
1,2,molasses,1


In [49]:
print(session.execute(q4).fetchone())

(1, 'dark chocolate chip', 1)


In [62]:
from sqlalchemy import MetaData
m = MetaData()
m.reflect(engine)

cook = m.tables[Cookie.__tablename__]

#print(session.execute(select(cook)).fetchall())
print(select(cook.c.cookie_id))

SELECT cookies.cookie_id 
FROM cookies


In [32]:
q2 = (
    text("SELECT cookie_id, cookie_name, quantity FROM cookies")
)

q22 = session.execute(q2).merge().fetchall()

d2 = dict(map(
    lambda _t: ((_t[1], 'True'), _t[2]),
    q22
))

print(d2)

{('dark chocolate chip', 'True'): 1, ('molasses', 'True'): 1}


In [37]:
print(session.scalars(func.sum(Cookie.quantity)).all())

[2]


In [36]:
q3 = text("SELECT COUNT(*) FROM cookies")
q33 = session.execute(q3).fetchone()[0]

print(q33)

2


In [5]:
print(type(q0))
print(type(q0.filter(Cookie.cookie_name=="dark chocolate chip")))

<class 'sqlalchemy.sql.selectable.Select'>
<class 'sqlalchemy.sql.selectable.Select'>


In [6]:
print(select(OrderType))

SELECT ordertype.ordertype_id, ordertype.item_sku, ordertype.item_type 
FROM ordertype


In [7]:
q2 = (
    select(Cookie, OrderType)
    .join(OrderType, Cookie.cookie_sku == OrderType.item_sku)
)

print(q2)

SELECT cookies.cookie_id, cookies.cookie_name, cookies.cookie_recipe_url, cookies.cookie_sku, cookies.quantity, cookies.unit_cost, ordertype.ordertype_id, ordertype.item_sku, ordertype.item_type 
FROM cookies JOIN ordertype ON cookies.cookie_sku = ordertype.item_sku


In [8]:
subq = q2.subquery()

print(subq.c.item_sku)

anon_1.item_sku


In [9]:
q = session.execute(select(Cookie))

print(q)

In [10]:
[x for x in session.scalars(select(Cookie)).all()]

[Cookie(cookie_name='dark chocolate chip', cookie_recipe_url='http://some.aweso.me/cookie/recipe_dark.html', cookie_sku='CC02', quantity=1, unit_cost=0.75),
 Cookie(cookie_name='molasses', cookie_recipe_url='http://some.aweso.me/cookie/recipe_molasses.html', cookie_sku='MOL01', quantity=1, unit_cost=0.8)]

In [11]:
result = engine.connect().execute(
    text("SELECT * FROM cookies")
)

print(result.first())

(1, 'dark chocolate chip', 'http://some.aweso.me/cookie/recipe_dark.html', 'CC02', 1, 0.75)


In [12]:
session.execute(text("SELECT * FROM cookies")).fetchall()

[(1, 'dark chocolate chip', 'http://some.aweso.me/cookie/recipe_dark.html', 'CC02', 1, 0.75),
 (2, 'molasses', 'http://some.aweso.me/cookie/recipe_molasses.html', 'MOL01', 1, 0.8)]

In [13]:
print(type(session.execute(text("SELECT * FROM cookies")).fetchall()))

<class 'list'>


In [14]:
engine.connect().execute(text("SELECT * FROM cookies")).fetchall()

[(1, 'dark chocolate chip', 'http://some.aweso.me/cookie/recipe_dark.html', 'CC02', 1, 0.75),
 (2, 'molasses', 'http://some.aweso.me/cookie/recipe_molasses.html', 'MOL01', 1, 0.8)]

In [15]:
session.execute(select(Cookie)).fetchall()

[(Cookie(cookie_name='dark chocolate chip', cookie_recipe_url='http://some.aweso.me/cookie/recipe_dark.html', cookie_sku='CC02', quantity=1, unit_cost=0.75),),
 (Cookie(cookie_name='molasses', cookie_recipe_url='http://some.aweso.me/cookie/recipe_molasses.html', cookie_sku='MOL01', quantity=1, unit_cost=0.8),)]

In [19]:
sess3 = sessionmaker(engine)
sess33 = sess3()

result = sess33.execute(select(Cookie))

print(result.fetchall())

[(Cookie(cookie_name='dark chocolate chip', cookie_recipe_url='http://some.aweso.me/cookie/recipe_dark.html', cookie_sku='CC02', quantity=1, unit_cost=0.75),), (Cookie(cookie_name='molasses', cookie_recipe_url='http://some.aweso.me/cookie/recipe_molasses.html', cookie_sku='MOL01', quantity=1, unit_cost=0.80),)]


In [35]:
#sess2 = session.begin()

with session.begin() as sess2:
        print(
                sess2.scalars(
                        select(func.count())
                        .select_from(Cookie)
                        .where(Cookie.cookie_sku=='MOL01')
                ).one()
        )

1


In [17]:
print(
    session.scalars(
            select(func.count())
            .select_from(Cookie)
            .where(Cookie.cookie_sku=='MOL01')
    ).one()
)

1


In [18]:
print(sqlalchemy.__version__)

2.0.3


In [31]:
with session.begin() as sess1:
    sess1.execute(
        update(Cookie)
        .values({Cookie.cookie_sku: None})
    )

    banana = sess1.execute(select(Cookie)).scalars().all()
    for x in banana:
        print(x)
    
    
    print(
        sess1.scalars(
                select(func.count())
                .select_from(Cookie)
                .where(Cookie.cookie_sku=='MOL01')
            ).one()
    )


    print(
        sess1.scalars(
                select(Cookie)
                .filter_by(cookie_name='dark chocolate chip')
            ).first().cookie_name
    )

Cookie(cookie_name='dark chocolate chip', cookie_recipe_url='http://some.aweso.me/cookie/recipe_dark.html', cookie_sku='None', quantity=1, unit_cost=0.75)
Cookie(cookie_name='molasses', cookie_recipe_url='http://some.aweso.me/cookie/recipe_molasses.html', cookie_sku='None', quantity=1, unit_cost=0.80)
0
dark chocolate chip


In [19]:
print(dcc.cookie_id)
print(mol.cookie_id)

1
2


In [24]:
current_cell_duplicate = "NJUDEK"
current_cell_duplicate = -1 if isinstance(current_cell_duplicate, str) else float(current_cell_duplicate)
print(current_cell_duplicate)

-1


In [30]:
from openpyxl import load_workbook

wb = load_workbook("MOCK_DATA.xlsx")


sheet = wb["data"]

print(float(sheet["Z1"].value))

TypeError: float() argument must be a string or a number, not 'NoneType'

In [25]:
x = "N/A"

y = x or 1

print(y)

N/A
